# Importing libraries

In [53]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import statistics as stats
import matplotlib.pyplot as plt
import tensorflow as tf

# Importing dataset

In [54]:
# train_dataset = pd.read_csv('../input/titanic/train.csv')
train_dataset = pd.read_csv('train.csv')
# print('for TRAINING SET : \n',train_dataset)
train_dataset.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [55]:
# test_dataset = pd.read_csv('test.csv')
# test_dataset = pd.read_csv('../input/titanic/test.csv')
# print('for TESTING SET : \n',test_dataset)

# test_dataset = pd.concat(map(pd.read_csv, ['gender_submission.csv', 'test.csv']), ignore_index=True)

test_dataset_X = pd.read_csv('test.csv')
test_dataset_y = pd.read_csv('gender_submission.csv')
test_dataset = pd.merge(test_dataset_y,test_dataset_X, on='PassengerId', how='inner')
test_dataset.head(10)
# print(len(test_dataset.values))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


# Preprocessing

In [56]:
# # Checking `null` values in dataset ; finding null values in columns
print('for TRAINING SET : \n',train_dataset.isnull().sum())
print('\nfor TESTING SET : \n',test_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

for TESTING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


> Missing values found : 
- training set : age, embarked
- testing set : age, cabin

## Filling missing values

- `Age` : 

In [57]:
train_dataset['Age'].fillna(train_dataset['Age'].mean(), inplace=True)
print('for TRAINING SET : \n',train_dataset.isnull().sum())
test_dataset['Age'].fillna(train_dataset['Age'].mean(), inplace=True)
print('for TRAINING SET : \n',test_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


- `Embarked` : 

In [58]:
train_dataset['Embarked'].fillna(train_dataset['Embarked'].mode()[0], inplace=True)
print('for TRAINING SET : \n',train_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


- `Fare` : 

In [59]:
test_dataset['Fare'].fillna(train_dataset['Fare'].mean(), inplace=True)
print('for TRAINING SET : \n',test_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


> considering only : `PClass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, `Embarked`

In [60]:
X_train = train_dataset.iloc[:,[2,4,5,6,7,9,11]].values
# X_train = train_dataset.iloc[:,[2,4,5,6,7,9,11]]
X_test = test_dataset.iloc[:,[2,4,5,6,7,9,11]].values
# X_test = test_dataset.iloc[:,[1,3,4,5,6,8,10]]
y_train = train_dataset.iloc[:,1].values
# y = train_dataset.iloc[:,1]
y_test = test_dataset.iloc[:,1].values

In [61]:
X_train

array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
       [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
       [3, 'female', 26.0, ..., 0, 7.925, 'S'],
       ...,
       [3, 'female', 29.69911764705882, ..., 2, 23.45, 'S'],
       [1, 'male', 26.0, ..., 0, 30.0, 'C'],
       [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object)

In [62]:
X_test

array([[3, 'male', 34.5, ..., 0, 7.8292, 'Q'],
       [3, 'female', 47.0, ..., 0, 7.0, 'S'],
       [2, 'male', 62.0, ..., 0, 9.6875, 'Q'],
       ...,
       [3, 'male', 38.5, ..., 0, 7.25, 'S'],
       [3, 'male', 29.699117647058763, ..., 0, 8.05, 'S'],
       [3, 'male', 29.699117647058763, ..., 1, 22.3583, 'C']],
      dtype=object)

In [63]:
y_train

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [64]:
y_test

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [65]:
# before encoding : 
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[3 'male' 22.0 1 0 7.25 'S']
0
[3 'male' 34.5 0 0 7.8292 'Q']
0


## Encoding categorical data

- encoding `gender` : 

In [66]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train[:, 1] = le.fit_transform(X_train[:, 1])
X_test[:, 1] = le.fit_transform(X_test[:, 1])

In [67]:
# after encoding gender : 
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[3 1 22.0 1 0 7.25 'S']
0
[3 1 34.5 0 0 7.8292 'Q']
0


- encoding `Embarked`

In [68]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.fit_transform(X_test))

In [69]:
# after encoding embarked : 
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[0.0 0.0 1.0 3 1 22.0 1 0 7.25]
0
[0.0 1.0 0.0 3 1 34.5 0 0 7.8292]
0


# Feature Scaling

In [70]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)   # only transforming to avoid Information Leakage

# Building the ANN

## 1. Initializing ANN

In [71]:
ann = tf.keras.models.Sequential()

## 2. Adding theinputlayer and the first hidden layer
- `units` : number of neurons in the layer
- `relu` : rectifier linear unit

In [72]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## 3. Adding the second hidden layer

In [73]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## 4. Adding the output layer
- as dimension of the output is 1 *(whether person is survived or not)*
- choosing sigmoid activation as we have binary classification

In [74]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Training the ANN

## 1. compiling the ANN
- here we are Updating weights and reducing cost.
- `optimizer` : gradient descent method. Here we are choosing adam as one of the method.
- `loss` : as we are doing binary classification, we'll use binary_crossentropy which will apply CROSS ENTROPY eq.
For non-binary classification use categorical_crossentropy as loss function and softmax as activation function above.
- `metrics` : choosing accuracy. although we can also choose multiple metrices.

In [75]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 2. Training the ANN on the training set

In [76]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
28/28 [==============================] - 1s 2ms/step - loss: 0.6393 - accuracy: 0.6263
Epoch 2/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6094 - accuracy: 0.6498
Epoch 3/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5839 - accuracy: 0.7149
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5611 - accuracy: 0.7565
Epoch 5/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5421 - accuracy: 0.7598
Epoch 6/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5255 - accuracy: 0.7666
Epoch 7/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5108 - accuracy: 0.7710
Epoch 8/100
28/28 [==============================] - 0s 2ms/step - loss: 0.4987 - accuracy: 0.7666
Epoch 9/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4885 - accuracy: 0.7710
Epoch 10/100
28/28 [==============================] - 0s 2ms/step - loss: 0.4805 - accuracy: 0.7677
Epoch 11/

# Making prediction and Model evaluation
- Prediction

In [77]:
y_pred = (ann.predict(X_test)>0.5)

- Evaluation through metrices

In [78]:
from sklearn.metrics import confusion_matrix, accuracy_score
print('Confusion Matrix :\n', confusion_matrix(y_test, y_pred))
print('Accuracy Score :', accuracy_score(y_test, y_pred))

Confusion Matrix :
 [[244  22]
 [ 17 135]]
Accuracy Score : 0.9066985645933014


- Making csv file for submission

In [79]:
test_dataset.PassengerId

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [80]:
predictions = []
for i in y_pred:
  if i[0]:
    predictions.append(1)
  else:
    predictions.append(0)
# predictions

In [81]:
output = pd.DataFrame({"PassengerId": test_dataset.PassengerId, "Survived": predictions})
print(output)
output.to_csv('19BCE245_DL_Prac1.csv', index=False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


# Extra models: 

## 1. XGBoost

In [82]:
from xgboost import XGBClassifier   # Similarly for regression, there is a class named as `XGRegressor` 
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[251  15]
 [ 31 121]]


0.8899521531100478

In [83]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print(f"Accuracy: {accuracies.mean()*100:.2f} %")
print(f"Standard Deviation: {accuracies.std()*100:.2f} %")

Accuracy: 82.27 %
Standard Deviation: 2.77 %


## 2. CatBoost

In [84]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.4 MB/s 


In [85]:
 from catboost import CatBoostClassifier   # The same CatBoostRegressor is available for Regression.
 classifier = CatBoostClassifier()
 classifier.fit(X_train, y_train)

Learning rate set to 0.009807
0:	learn: 0.6865383	total: 48.1ms	remaining: 48.1s
1:	learn: 0.6806346	total: 48.7ms	remaining: 24.3s
2:	learn: 0.6767701	total: 49.5ms	remaining: 16.5s
3:	learn: 0.6709615	total: 50.7ms	remaining: 12.6s
4:	learn: 0.6650482	total: 52ms	remaining: 10.3s
5:	learn: 0.6590147	total: 53.3ms	remaining: 8.82s
6:	learn: 0.6533568	total: 54.5ms	remaining: 7.73s
7:	learn: 0.6478691	total: 55.8ms	remaining: 6.92s
8:	learn: 0.6428664	total: 56.7ms	remaining: 6.24s
9:	learn: 0.6370185	total: 58ms	remaining: 5.74s
10:	learn: 0.6325716	total: 59.2ms	remaining: 5.32s
11:	learn: 0.6281765	total: 60.1ms	remaining: 4.95s
12:	learn: 0.6232064	total: 61.4ms	remaining: 4.66s
13:	learn: 0.6187835	total: 62.6ms	remaining: 4.41s
14:	learn: 0.6137314	total: 63.8ms	remaining: 4.19s
15:	learn: 0.6092345	total: 65.1ms	remaining: 4s
16:	learn: 0.6056495	total: 66ms	remaining: 3.81s
17:	learn: 0.6004947	total: 67.3ms	remaining: 3.67s
18:	learn: 0.5964621	total: 68.5ms	remaining: 3.54s
1

In [86]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[246  20]
 [ 33 119]]


0.8732057416267942

In [87]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print(f"Accuracy: {accuracies.mean()*100:.2f} %")
print(f"Standard Deviation: {accuracies.std()*100:.2f} %")

Streaming output truncated to the last 5000 lines.
6:	learn: 0.6558006	total: 17.5ms	remaining: 2.48s
7:	learn: 0.6499328	total: 20.7ms	remaining: 2.57s
8:	learn: 0.6446133	total: 24.1ms	remaining: 2.66s
9:	learn: 0.6394966	total: 28ms	remaining: 2.77s
10:	learn: 0.6361806	total: 31.3ms	remaining: 2.81s
11:	learn: 0.6316201	total: 32.6ms	remaining: 2.69s
12:	learn: 0.6268850	total: 33.7ms	remaining: 2.56s
13:	learn: 0.6221803	total: 35ms	remaining: 2.47s
14:	learn: 0.6174166	total: 36.4ms	remaining: 2.39s
15:	learn: 0.6135923	total: 37.3ms	remaining: 2.29s
16:	learn: 0.6087086	total: 38.4ms	remaining: 2.22s
17:	learn: 0.6040934	total: 53.2ms	remaining: 2.9s
18:	learn: 0.6003858	total: 55.3ms	remaining: 2.86s
19:	learn: 0.5960078	total: 58.1ms	remaining: 2.85s
20:	learn: 0.5917568	total: 61.2ms	remaining: 2.85s
21:	learn: 0.5895572	total: 61.7ms	remaining: 2.74s
22:	learn: 0.5854647	total: 63.1ms	remaining: 2.68s
23:	learn: 0.5813575	total: 64.4ms	remaining: 2.62s
24:	learn: 0.5769566	t

# Training various models on the Training set

### 1. Logistic Regression

In [88]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0)
clf_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [89]:
y_pred_lr = clf_lr.predict(X_test)

### 2. K-Nearest Neighbor (K-NN)

In [90]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
clf_knn.fit(X_train, y_train)

KNeighborsClassifier()

In [91]:
y_pred_knn = clf_knn.predict(X_test)

### 3. Support Vector Machine (SVM)

In [92]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear', random_state=0)
clf_svc.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [93]:
y_pred_svc = clf_svc.predict(X_test)

### 4. Kernel SVM

In [94]:
from sklearn.svm import SVC
clf_kernelSVC = SVC(kernel='rbf', random_state=0)
clf_kernelSVC.fit(X_train, y_train)

SVC(random_state=0)

In [95]:
y_pred_kernelSVC = clf_kernelSVC.predict(X_test)

### 5. Naïve Bayes

In [96]:
from sklearn.naive_bayes import GaussianNB
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)

GaussianNB()

In [97]:
y_pred_nb = clf_nb.predict(X_test)

### 6. Decision Tree

  #### 6.1 with **GINI**

In [98]:
from sklearn.tree import DecisionTreeClassifier
clf_dtGINI = DecisionTreeClassifier(criterion='gini',  random_state=0)
clf_dtGINI.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [99]:
y_pred_dtGINI = clf_dtGINI.predict(X_test)

  #### 6.2 with **ENTROPY**

In [100]:
from sklearn.tree import DecisionTreeClassifier
clf_dtENTROPY = DecisionTreeClassifier(criterion='entropy',  random_state=0)
clf_dtENTROPY.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [101]:
y_pred_dtENTROPY = clf_dtENTROPY.predict(X_test)

### 7. Random Forest Classifier

  #### 7.2 with **ENTROPY**

In [102]:
from sklearn.ensemble import RandomForestClassifier
clf_rfcGINI = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 0)
clf_rfcGINI.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

In [103]:
y_pred_rfcGINI = clf_rfcGINI.predict(X_test)

  #### 7.2 with **ENTROPY**

In [104]:
from sklearn.ensemble import RandomForestClassifier
clf_rfcENTROPY = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
clf_rfcENTROPY.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [105]:
y_pred_rfcENTROPY = clf_rfcENTROPY.predict(X_test)

### Evaluating the model performance with Confusion Matrix

In [106]:
pip install -U prettytable

In [107]:
from prettytable import PrettyTable
from sklearn.metrics import confusion_matrix, accuracy_score

In [108]:
evaluataionTable = PrettyTable()
evaluataionTable.field_names = ["Model", "Confusion Matrix", "Accuracy"]
evaluataionTable.add_row(["Logistic Regression", confusion_matrix(y_test, y_pred_lr), accuracy_score(y_test, y_pred_lr)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["K Nearest Neighbor", confusion_matrix(y_test, y_pred_knn), accuracy_score(y_test, y_pred_knn)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["Support Vector Machine", confusion_matrix(y_test, y_pred_svc), accuracy_score(y_test, y_pred_svc)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["SVM Kernel", confusion_matrix(y_test, y_pred_kernelSVC), accuracy_score(y_test, y_pred_kernelSVC)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["Naïve Bayes", confusion_matrix(y_test, y_pred_nb), accuracy_score(y_test, y_pred_nb)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["Decision Tree (with GINI)", confusion_matrix(y_test, y_pred_dtGINI), accuracy_score(y_test, y_pred_dtGINI)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["Decision Tree (with Entropy)", confusion_matrix(y_test, y_pred_dtENTROPY), accuracy_score(y_test, y_pred_dtENTROPY)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["Random Forest (with GINI)", confusion_matrix(y_test, y_pred_rfcGINI), accuracy_score(y_test, y_pred_rfcGINI)])
evaluataionTable.add_row(["------------------------------", "------------------", "--------------------"])
evaluataionTable.add_row(["Random Forest (with ENTROPY)", confusion_matrix(y_test, y_pred_rfcENTROPY), accuracy_score(y_test, y_pred_rfcENTROPY)])
print(evaluataionTable)

+--------------------------------+--------------------+----------------------+
|             Model              |  Confusion Matrix  |       Accuracy       |
+--------------------------------+--------------------+----------------------+
|      Logistic Regression       |     [[253  13]     |  0.9449760765550239  |
|                                |     [ 10 142]]     |                      |
| ------------------------------ | ------------------ | -------------------- |
|       K Nearest Neighbor       |     [[231  35]     |  0.8588516746411483  |
|                                |     [ 24 128]]     |                      |
| ------------------------------ | ------------------ | -------------------- |
|     Support Vector Machine     |     [[266   0]     |         1.0          |
|                                |     [  0 152]]     |                      |
| ------------------------------ | ------------------ | -------------------- |
|           SVM Kernel           |     [[256  10]   

### Confusion Matrix : 

In [109]:
confusionMatrixTable = PrettyTable()
confusionMatrixTable.field_names = ["Model", "Accuracy"]
confusionMatrixTable.add_row(["Logistic Regression", confusion_matrix(y_test, y_pred_lr)])
confusionMatrixTable.add_row(["K Nearest Neighbor", confusion_matrix(y_test, y_pred_knn)])
confusionMatrixTable.add_row(["Support Vector Machine", confusion_matrix(y_test, y_pred_svc)])
confusionMatrixTable.add_row(["SVM Kernel", confusion_matrix(y_test, y_pred_kernelSVC)])
confusionMatrixTable.add_row(["Naïve Bayes", confusion_matrix(y_test, y_pred_nb)])
confusionMatrixTable.add_row(["Decision Tree (with GINI)", confusion_matrix(y_test, y_pred_dtGINI)])
confusionMatrixTable.add_row(["Decision Tree (with Entropy)", confusion_matrix(y_test, y_pred_dtENTROPY)])
confusionMatrixTable.add_row(["Random Forest (with GINI)", confusion_matrix(y_test, y_pred_rfcGINI)])
confusionMatrixTable.add_row(["Random Forest (with ENTROPY)", confusion_matrix(y_test, y_pred_rfcENTROPY)])
print(confusionMatrixTable)

+------------------------------+-------------+
|            Model             |   Accuracy  |
+------------------------------+-------------+
|     Logistic Regression      |  [[253  13] |
|                              |  [ 10 142]] |
|      K Nearest Neighbor      |  [[231  35] |
|                              |  [ 24 128]] |
|    Support Vector Machine    |  [[266   0] |
|                              |  [  0 152]] |
|          SVM Kernel          |  [[256  10] |
|                              |  [ 31 121]] |
|         Naïve Bayes          |  [[223  43] |
|                              |  [ 12 140]] |
|  Decision Tree (with GINI)   |  [[218  48] |
|                              |  [ 36 116]] |
| Decision Tree (with Entropy) |  [[221  45] |
|                              |  [ 37 115]] |
|  Random Forest (with GINI)   |  [[237  29] |
|                              |  [ 36 116]] |
| Random Forest (with ENTROPY) |  [[234  32] |
|                              |  [ 40 112]] |
+------------

### Accuracy Table : 

In [110]:
AccuracyTable = PrettyTable()
AccuracyTable.field_names = ["Model", "Accuracy"]
AccuracyTable.add_row(["Logistic Regression", accuracy_score(y_test, y_pred_lr)])
AccuracyTable.add_row(["K Nearest Neighbor", accuracy_score(y_test, y_pred_knn)])
AccuracyTable.add_row(["Support Vector Machine", accuracy_score(y_test, y_pred_svc)])
AccuracyTable.add_row(["SVM Kernel", accuracy_score(y_test, y_pred_kernelSVC)])
AccuracyTable.add_row(["Naïve Bayes", accuracy_score(y_test, y_pred_nb)])
AccuracyTable.add_row(["Decision Tree (with GINI)", accuracy_score(y_test, y_pred_dtGINI)])
AccuracyTable.add_row(["Decision Tree (with Entropy)", accuracy_score(y_test, y_pred_dtENTROPY)])
AccuracyTable.add_row(["Random Forest (with GINI)", accuracy_score(y_test, y_pred_rfcGINI)])
AccuracyTable.add_row(["Random Forest (with ENTROPY)", accuracy_score(y_test, y_pred_rfcENTROPY)])
print(AccuracyTable)

+------------------------------+--------------------+
|            Model             |      Accuracy      |
+------------------------------+--------------------+
|     Logistic Regression      | 0.9449760765550239 |
|      K Nearest Neighbor      | 0.8588516746411483 |
|    Support Vector Machine    |        1.0         |
|          SVM Kernel          | 0.9019138755980861 |
|         Naïve Bayes          | 0.868421052631579  |
|  Decision Tree (with GINI)   | 0.7990430622009569 |
| Decision Tree (with Entropy) | 0.8038277511961722 |
|  Random Forest (with GINI)   | 0.8444976076555024 |
| Random Forest (with ENTROPY) | 0.8277511961722488 |
+------------------------------+--------------------+


# Grid Search with SVM
- taking too much load.
> IGNORE THIS SECTION.

In [111]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[256  10]
 [ 31 121]]


0.9019138755980861

In [112]:
# k-fold
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)
print(f"Accuracy : {accuracies.mean()*100:.2f}%")
print(f"Standard Deviation : {accuracies.std()*100:.2f}%")

Accuracy : 82.49%
Standard Deviation : 3.56%


In [113]:
#grid search 
from sklearn.model_selection import GridSearchCV
parameters = [
             {
                 'C' : np.linspace(0.01, 1,num=10).tolist(),    # [0, 0.25, 0.5, 0.75, 1]
                 'kernel' : ['linear'],
                 'degree' : [3, 4, 5],
             },
             {
                 'C' : np.linspace(0.01, 1,num=10).tolist(),    # [0, 0.25, 0.5, 0.75, 1]
                 'kernel' : ['rbf','poly','sigmoid'],
                 'gamma' : np.linspace(0.01, 1,num=10).tolist(), # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
                 'degree' : [3,4,5],
             }
            ]
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=parameters,
                           scoring='accuracy',  # As we are doing classification, we are using accuracy for scoring models
                           cv=10, # number of folds (same like k-fold cross validation)
                           n_jobs=-1 # all your processors will be used available in hardware
                           )
grid_search.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
#grid search 
from sklearn.model_selection import GridSearchCV
parameters = [
            #  {
            #      'C' :  [0, 0.25, 0.5, 0.75, 1],
            #      'kernel' : ['linear'],
            #      'degree' : [3, 4, 5],
            #  },
             {
                 'C' :  [0, 0.3, 0.65, 1],
                 'kernel' : ['rbf','poly','sigmoid'],
                 'gamma' :  [0.1,  0.5, 0.9],
                 'degree' : [3, 4, 5],
             }
            ]
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=parameters,
                           scoring='accuracy',  # As we are doing classification, we are using accuracy for scoring models
                           cv=4, # number of folds (same like k-fold cross validation)
                           n_jobs=-1 # all your processors will be used available in hardware
                           )
grid_search.fit(X_train, y_train)

In [ ]:
best_accuracy = grid_search.best_score_
print(f"Best Accuracy achieved : {best_accuracy*100:.2f}%")

In [ ]:
best_parameters = grid_search.best_params_
print(f"Best parameters achieved : {best_parameters}")